In [ ]:
!pip install openai langchain

In [ ]:
!pip install -U langchain-openai

In [ ]:
import openai
import langchain
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI

In [ ]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 4.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


Extracting OpenAI api key from my google drive

In [ ]:
with open('/content/drive/My Drive/openai_api_key.txt', 'r') as file:
    apikey = file.readline().strip()


In [ ]:
!pip install pinecone-client

We are using OpenAI Embeddings for converting out input text into embeddings

In [ ]:
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings(api_key=apikey)
embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x7c1e8c1dff70>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7c1e7ff5c340>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version='', openai_api_base=None, openai_api_type='', openai_proxy='', embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=set(), disallowed_special='all', chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

We are using pre-embedding dataset of langchain docs

In [ ]:
from pinecone_datasets import load_dataset

dataset = load_dataset('langchain-python-docs-text-embedding-ada-002')
# we drop sparse_values as they are not needed for this example
dataset.documents.drop(['metadata'], axis=1, inplace=True)
dataset.documents.rename(columns={'blob': 'metadata'}, inplace=True)
dataset.head()

,id,values,sparse_values,metadata
0.0,417ede5d-39be-498f-b518-f47ed4e53b90,"[0.005949743557721376, 0.01983247883617878, -0...",None,"{'chunk': 0, 'text': '.rst .pdf Welcome to Lan..."
1.0,110f550d-110b-4378-b95e-141397fa21bc,"[0.009401749819517136, 0.02443608082830906, 0....",None,"{'chunk': 1, 'text': 'Use Cases# Best practice..."
2.0,d5f00f02-3295-4567-b297-5e3262dc2728,"[-0.005517194513231516, 0.0208403542637825, 0....",None,"{'chunk': 2, 'text': 'Gallery: A collection of..."
3.0,0b6fe3c6-1f0e-4608-a950-43231e46b08a,"[-0.006499645300209522, 0.0011573900701478124,...",None,"{'chunk': 0, 'text': 'Search Error Please acti..."
4.0,39d5f15f-b973-42c0-8c9b-a2df49b627dc,"[-0.005658374633640051, 0.00817849114537239, 0...",None,"{'chunk': 0, 'text': '.md .pdf Dependents Depe..."


Initialization of Pinecone and inserting the values into the vector database

In [ ]:
from pinecone import Pinecone

pc = Pinecone(api_key="cde0cc99-ae70-404d-bca8-63f3d8057d99")
index = pc.Index("congressmanifesto")

In [ ]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [ ]:
for batch in dataset.iter_documents(batch_size=100):
    index.upsert(batch)

In [ ]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 3476}},
 'total_vector_count': 3476}

In [ ]:
!pip install pinecone_datasets

In [ ]:
query = "how to use pinecone in langchain?"


# retrieve from Pinecone
xq = embeddings.embed_query(query)

# get relevant contexts (including the questions)
res = index.query(vector=xq, top_k=5, include_metadata=True)

In [ ]:
res

{'matches': [{'id': '31882455-5eb6-42a4-bdf0-98ab8a7ca259',
              'metadata': {'chunk': 8.0,
                           'text': 'LangChain is a framework for including AI '
                                   'from large language models inside data '
                                   'pipelines and applications. This tutorial '
                                   'provides an overview of what you ... '
                                   'LangChain is an intuitive framework '
                                   'created to assist in developing '
                                   'applications driven by a language model, '
                                   'such as OpenAI or Hugging Face. This '
                                   'documentation covers the steps to '
                                   'integrate Pinecone, a high-performance '
                                   'vector database, with LangChain, a '
                                   'framework for building applicat

In [ ]:
# get list of retrieved text
contexts = [item['metadata']['text'] for item in res['matches']]

augmented_query = "\n\n---\n\n".join(contexts)+"\n\n-----\n\n"+query

In [ ]:
print(augmented_query)

LangChain is a framework for including AI from large language models inside data pipelines and applications. This tutorial provides an overview of what you ... LangChain is an intuitive framework created to assist in developing applications driven by a language model, such as OpenAI or Hugging Face. This documentation covers the steps to integrate Pinecone, a high-performance vector database, with LangChain, a framework for building applications powered ... The ability to connect to any model, ingest any custom database, and build upon a framework that can take action provides numerous use cases for ... With LangChain, developers can use a framework that abstracts the core building blocks of LLM applications. LangChain empowers developers to ... Build a question-answering tool based on financial data with LangChain & Deep Lake's unified & streamable data store. Browse applications built on LangChain technology. Explore PoC and MVP applications created by our community and discover inno

Setting up our Q&A chatbot

In [ ]:

primer = f"""You are Q&A bot. A highly intelligent system that answers
user questions based on the information provided by the user above
each question. If the information can not be found in the information
provided by the user you truthfully say "I don't know".
"""


from openai import OpenAI

client = OpenAI(

    api_key=apikey,
)

chat_completion = client.chat.completions.create(
    messages=[
        {"role": "system", "content": primer},
        {"role": "user", "content": augmented_query}
    ],
    model="gpt-3.5-turbo",
)

In [ ]:
from IPython.display import Markdown

display(Markdown(chat_completion.choices[0].message.content))

To use Pinecone in LangChain, you can follow these steps:

1. **Installation and Setup**: Install the Python SDK with `pip install pinecone-client`.
   
2. **Wrappers**: There exists a wrapper around Pinecone indexes, allowing you to use it as a vector store, whether for semantic search or example selection.
   
    To import this vector store in LangChain, you can use the following code snippet:
   
    ```python
    from langchain.vectorstores import Pinecone
    ```

For a more detailed walkthrough of using the Pinecone wrapper within LangChain, you can refer to the notebook mentioned in the documentation.

If you need further assistance or more information on using Pinecone with LangChain, feel free to ask!